# Notebook for Preprocessing Dataset on Fuzzy Project

In [1]:
import string
import numpy as np
import pandas as pd

# Preprocessing Functions

In [2]:
raw_path = '../dataset/raw/Data Fuzzy (Jawaban).xlsx'

def read_raw():
    raw = pd.read_excel(raw_path, sheet_name='raw', index_col=[0])
    raw.columns = raw.columns.str.lower()
    raw.index = raw.index.str.lower()
    for col in raw.columns:
        raw[col] = raw[col].str.lower()
    return raw

def get_mapping(people):
    people = sorted(list(map(lambda x: x.lower(), people)))
    map_val = list(string.ascii_uppercase[:len(people)])

    name2char = dict(zip(people, map_val))
    char2name = dict(zip(map_val, people))

    return name2char, char2name

def get_masked(df_, mapping):
    df = df_.copy()
    df.columns.name = 'person'

    for col in df.columns:
        for k, v in mapping.items():
            df[col] = df[col].str.replace(k, v)
    
    output_rep = {f'output [{x}]': f'output [{mapping[x]}]' for x in mapping}
    df.index = df.index.to_frame().person.replace(mapping).values
    df.columns = df.columns.to_frame().person.replace(output_rep).values
    df.index.name = 'person'

    return df

def count_skill(skill_, min_val=None, max_val=None):
    people = sorted(list(skill_.index))
    skill = pd.Series(0, index=people)
    for person, group in skill_.items():
        group = group.split(', ')
        skill[group] += 1

    mu = skill / len(people)
    if min_val is None:
        min_val = skill.min()
    if max_val is None:
        max_val = skill.max()
    mu = (max_val - min_val) * (mu - mu.min()) / (mu.max() - mu.min()) + min_val

    skill.index.name = 'person'
    mu.index.name = 'person'
    skill.name = 'skill_vote'
    mu.name = 'mu'
    return skill, mu

def get_group_freq(group_data):
    people = sorted(list(group_data.index))
    freq = pd.DataFrame(0, index=people, columns=people)
    freq.index.name = 'person'

    for col in group_data.columns:
        groups = group_data[col].str.split(', ')
        for person, group in groups.items():
            if isinstance(group, float):
                # means group is null/NaN
                continue
            freq.loc[person, group] += 1

    mu_freq = freq / group_data.notnull().sum(axis=1)
    assert((freq.values == freq.values.T).all())
    return freq, mu_freq

def fuzzify_output(vote_data):
    people = sorted(list(vote_data.index))
    output = pd.DataFrame(0, index=people, columns=people)
    output.index.name = 'person'

    for _, votes in vote_data.iterrows():
        temp_vote = pd.DataFrame(0, index=people, columns=people)
        for person, choice in votes.items():
            if isinstance(choice, float):
                # means choice is null/NaN
                continue
            choice = choice.split(', ')
            if person not in choice:
                choice.append(person)
            temp_vote.loc[choice, choice] = 1

        output += temp_vote

    np.fill_diagonal(output.values, 0)
    assert((output.values == output.values.T).all())

    mu_output = output.copy() / len(people)
    
    return output, mu_output

# Read Data and Masking

In [3]:
raw = read_raw()
people = raw.index.values
name2char, char2name = get_mapping(people)

raw = get_masked(raw, name2char)
raw.head(3)

,skill,output [E],output [H],output [G],output [C],output [M],output [J],output [D],output [B],output [F],...,kelompok_mrka,kelompok_ml_1,kelompok_ml_2,kelompok_spk,kelompok_sispak,kelompok_pekom,kelompok_stmh_1,kelompok_fuzzy,kelompok_cv,kelompok_pka
person,,,,,,,,,,,,,,,,,,,,,
E,"H, C, J, L, D, I",NaN,NaN,"M, J",NaN,"G, J","G, M",NaN,NaN,NaN,...,"D, F",L,"H, G",NaN,NaN,"H, F",NaN,"C, J","L, I, M","C, F, J"
K,"E, H, D, F, L",NaN,NaN,"M, J",NaN,"G, J","G, M",NaN,NaN,NaN,...,"C, B",C,"A, L, I",NaN,NaN,NaN,"H, I","B, L","B, C","G, M, I, B"
F,"E, H, D, L, I, K","D, L, I",NaN,"M, J",NaN,"G, J","G, M","E, L, I",NaN,NaN,...,"E, D",H,"D, B",NaN,"J, C","H, E",NaN,"G, H, D",NaN,"E, C, J"


# Preprocess Input 1: Skill

In [4]:
skill, mu_skill = count_skill(raw.skill, min_val=0.5, max_val=1.0)
mu_skill

person
A    0.500000
B    0.545455
C    0.590909
D    1.000000
E    1.000000
F    0.727273
G    0.500000
H    0.909091
I    1.000000
J    0.590909
K    0.636364
L    1.000000
M    0.545455
Name: mu, dtype: float64

# Preprocess Input 2: Frequency

## Pre-mid Frequency

In [5]:
group_cols = raw.columns[raw.columns.str.startswith('kelompok')]
premid_group_cols = [0, 1, 3, 4, 5, 6]
premid_group_cols = group_cols[premid_group_cols]

print('All    :', group_cols.tolist())
print('Pre-mid:', premid_group_cols.tolist())

All    : ['kelompok_mrka', 'kelompok_ml_1', 'kelompok_ml_2', 'kelompok_spk', 'kelompok_sispak', 'kelompok_pekom', 'kelompok_stmh_1', 'kelompok_fuzzy', 'kelompok_cv', 'kelompok_pka']
Pre-mid: ['kelompok_mrka', 'kelompok_ml_1', 'kelompok_spk', 'kelompok_sispak', 'kelompok_pekom', 'kelompok_stmh_1']


In [6]:
group_data = raw[premid_group_cols]
group_data

,kelompok_mrka,kelompok_ml_1,kelompok_spk,kelompok_sispak,kelompok_pekom,kelompok_stmh_1
person,,,,,,
E,"D, F",L,NaN,NaN,"H, F",NaN
K,"C, B",C,NaN,NaN,NaN,"H, I"
F,"E, D",H,NaN,"J, C","H, E",NaN
J,"G, M","G, M",NaN,"C, F",NaN,"A, L"
D,"E, F",B,"G, M, B",NaN,NaN,NaN
G,"M, J","M, J","M, B, D",NaN,NaN,NaN
M,"J, G","G, J","G, D, B",NaN,NaN,NaN
I,"H, L, A",A,NaN,NaN,NaN,"K, H"
B,"C, K",D,"G, M, D",NaN,NaN,NaN


In [7]:
premid_group_freq, mu_premid_group = get_group_freq(group_data)
premid_group_freq

,A,B,C,D,E,F,G,H,I,J,K,L,M
person,,,,,,,,,,,,,
A,0,0,0,0,0,0,0,1,2,1,0,2,0
B,0,0,1,2,0,0,1,0,0,0,1,0,1
C,0,1,0,0,0,1,0,0,0,1,2,0,0
D,0,2,0,0,1,1,1,0,0,0,0,0,1
E,0,0,0,1,0,2,0,1,0,0,0,1,0
F,0,0,1,1,2,0,0,2,0,1,0,0,0
G,0,1,0,1,0,0,0,0,0,2,0,0,3
H,1,0,0,0,1,2,0,0,2,0,1,1,0
I,2,0,0,0,0,0,0,2,0,0,1,1,0


## All Frequency

In [8]:
group_data = raw[group_cols]
group_freq, mu_group_freq = get_group_freq(group_data)
group_freq

,A,B,C,D,E,F,G,H,I,J,K,L,M
person,,,,,,,,,,,,,
A,0,0,0,2,0,0,1,2,4,1,1,4,1
B,0,0,2,3,0,1,2,0,1,0,4,1,2
C,0,2,0,0,2,2,0,0,0,4,3,0,1
D,2,3,0,0,1,3,3,2,0,0,0,1,1
E,0,0,2,1,0,3,1,2,1,2,0,2,1
F,0,1,2,3,3,0,1,3,0,2,0,0,0
G,1,2,0,3,1,1,0,2,1,2,1,0,4
H,2,0,0,2,2,3,2,0,2,0,1,2,0
I,4,1,0,0,1,0,1,2,0,0,3,3,3


# Preprocess Output

In [9]:
output_cols = raw.columns[raw.columns.str.startswith('output')]
output_data = raw[output_cols]
output_data.columns = output_data.columns.str.extract('output \[(.*?)\]', expand=False)

output, mu_output = fuzzify_output(output_data)
output

,A,B,C,D,E,F,G,H,I,J,K,L,M
person,,,,,,,,,,,,,
A,0,1,2,1,0,2,2,2,0,3,1,1,1
B,1,0,3,1,1,1,1,1,2,1,3,2,1
C,2,3,0,1,1,1,1,1,1,1,3,3,1
D,1,1,1,0,7,4,1,7,6,1,2,7,0
E,0,1,1,7,0,3,0,5,7,0,2,9,0
F,2,1,1,4,3,0,2,5,3,3,1,4,1
G,2,1,1,1,0,2,0,3,0,11,1,1,11
H,2,1,1,7,5,5,3,0,5,3,1,6,2
I,0,2,1,6,7,3,0,5,0,0,2,7,0


In [10]:
mu_output

,A,B,C,D,E,F,G,H,I,J,K,L,M
person,,,,,,,,,,,,,
A,0.000000,0.076923,0.153846,0.076923,0.000000,0.153846,0.153846,0.153846,0.000000,0.230769,0.076923,0.076923,0.076923
B,0.076923,0.000000,0.230769,0.076923,0.076923,0.076923,0.076923,0.076923,0.153846,0.076923,0.230769,0.153846,0.076923
C,0.153846,0.230769,0.000000,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.230769,0.230769,0.076923
D,0.076923,0.076923,0.076923,0.000000,0.538462,0.307692,0.076923,0.538462,0.461538,0.076923,0.153846,0.538462,0.000000
E,0.000000,0.076923,0.076923,0.538462,0.000000,0.230769,0.000000,0.384615,0.538462,0.000000,0.153846,0.692308,0.000000
F,0.153846,0.076923,0.076923,0.307692,0.230769,0.000000,0.153846,0.384615,0.230769,0.230769,0.076923,0.307692,0.076923
G,0.153846,0.076923,0.076923,0.076923,0.000000,0.153846,0.000000,0.230769,0.000000,0.846154,0.076923,0.076923,0.846154
H,0.153846,0.076923,0.076923,0.538462,0.384615,0.384615,0.230769,0.000000,0.384615,0.230769,0.076923,0.461538,0.153846
I,0.000000,0.153846,0.076923,0.461538,0.538462,0.230769,0.000000,0.384615,0.000000,0.000000,0.153846,0.538462,0.000000


# Save It All

In [11]:
raw.to_csv('../dataset/clean/clean.csv')
skill.to_csv('../dataset/clean/skill.csv')
mu_skill.to_csv('../dataset/clean/mu_skill.csv')
group_freq.to_csv('../dataset/clean/freq.csv')
mu_group_freq.to_csv('../dataset/clean/mu_freq.csv')
premid_group_freq.to_csv('../dataset/clean/premid_freq.csv')
mu_premid_group.to_csv('../dataset/clean/mu_premid_freq.csv')
output.to_csv('../dataset/clean/output.csv')
mu_output.to_csv('../dataset/clean/mu_output.csv')